Install and load all dependencies (first time only) \
NOTE: you may need to restart the runtime afterwards (CTRL+M .).

In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-glx libglew-dev libglew2.0 libosmesa6 libosmesa6-dev
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,916 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgl1-mesa-glx amd64 20.0.8-0ubuntu1~18.04.1 [5,532 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglew2.0 amd64 2.0.0-5 [140 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/univ

Set up the custom Hopper environment



1.   Upload `classes.zip` to the current session's file storage
2.   Un-zip it by running cell below


In [3]:
!unzip classes.zip

Archive:  classes.zip
  inflating: classes/actor_critic.py  
  inflating: classes/data.npy        
  inflating: classes/dropo.py        
   creating: classes/env/
 extracting: classes/env/__init__.py  
   creating: classes/env/__pycache__/
  inflating: classes/env/__pycache__/__init__.cpython-38.pyc  
  inflating: classes/env/__pycache__/custom_hopper.cpython-38.pyc  
  inflating: classes/env/__pycache__/jinja_mujoco_env.cpython-38.pyc  
  inflating: classes/env/__pycache__/mujoco_env.cpython-38.pyc  
  inflating: classes/env/__pycache__/random_mass_hopper.cpython-38.pyc  
  inflating: classes/env/__pycache__/template_renderer.cpython-38.pyc  
   creating: classes/env/assets/
  inflating: classes/env/assets/hopper.xml  
  inflating: classes/env/custom_hopper.py  
  inflating: classes/env/mujoco_env.py  
  inflating: classes/vanilla.py      


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




---



\

**Test an RL agent on the OpenAI Gym Hopper environment**

\
Choose which agent you want to test.


In [4]:
import torch
import gym

from classes.env.custom_hopper import *
#from classes.vanilla import Agent, Policy
from classes.actor_critic import Agent, Policy
import numpy as np
import matplotlib.pyplot as plt

Compiling /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.7/dist-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuextensionbuilder/temp.linux-x86_64-3.7/usr/local/lib
creating /usr/local/lib/python3.7/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_37_linuxcpuexten

In [12]:
model = 'model.mdl' # Fill in model path
device = 'cpu'
episodes = 50

In [13]:
#@title
#env = gym.make('CustomHopper-source-v0')
env = gym.make('CustomHopper-target-v0')

print('Action space:', env.action_space)
print('State space:', env.observation_space)
print('Dynamics parameters:', env.get_parameters())

observation_space_dim = env.observation_space.shape[-1]
action_space_dim = env.action_space.shape[-1]

policy = Policy(observation_space_dim, action_space_dim)
policy.load_state_dict(torch.load(model), strict=True)

agent = Agent(1e-4, 0.99, policy, device=device)

Action space: Box([-1. -1. -1.], [1. 1. 1.], (3,), float32)
State space: Box([-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf inf inf inf inf], (11,), float64)
Dynamics parameters: [3.53429174 3.92699082 2.71433605 5.0893801 ]


In [14]:
test_rewards = []
for episode in range(episodes):
  done = False
  test_reward = 0
  state = env.reset()

  while not done:
    action, _ = agent.get_action(state, evaluation=True)
    
    state, reward, done, info = env.step(action.detach().cpu().numpy())

    test_reward += reward
  test_rewards.append(test_reward)
  print(f"Episode: {episode} | Return: {test_reward}")

Episode: 0 | Return: -984.0613348349127
Episode: 1 | Return: -983.7845646755713
Episode: 2 | Return: -984.0054576095275
Episode: 3 | Return: -984.007210890793
Episode: 4 | Return: -983.977363678695
Episode: 5 | Return: -984.0052689043943
Episode: 6 | Return: -983.7944241809973
Episode: 7 | Return: -983.7924566678046
Episode: 8 | Return: -983.7068878041098
Episode: 9 | Return: -983.9186849917108
Episode: 10 | Return: -983.806278235724
Episode: 11 | Return: -983.7481112908617
Episode: 12 | Return: -983.9215500334833
Episode: 13 | Return: -984.0815970586748
Episode: 14 | Return: -983.9991453296781
Episode: 15 | Return: -983.8166160473234
Episode: 16 | Return: -984.0101818897357
Episode: 17 | Return: -983.7765847320866
Episode: 18 | Return: -983.9934939344961
Episode: 19 | Return: -984.0315026202081
Episode: 20 | Return: -983.8445749140133
Episode: 21 | Return: -983.9325346949479
Episode: 22 | Return: -983.9793590422128
Episode: 23 | Return: -984.0371671637151
Episode: 24 | Return: -983.70

In [15]:
avg = np.sum(test_rewards)/len(test_rewards)
avg

-983.8919678736343